Credits: Dieses Notebook baut auf Code/Codebausteinen von Hr. Martin Cornejo (TUM) auf 
Das Aufgaben-Notebook wurde für die Vorlesung STAT-bat im SoSe2023 an der HS Kempten von Prof. Dr. Holger Hesse zur Verfügung gestellt.


# NB1: Betriebsstrategien für Heimspeichersysteme

Heimspeichersysteme dienen der Eigenverbrauchserhöhung in Privathaushalten, bei denen eine lokale Erzeugungsanlage (meist: Photovoltaik) einen Teil der Stromversorgung deckt. 

Aus Kundensicht können neben monetären Faktoren auch ein Bestreben nach "mehr Autrarkie" oder einer mögl. positiven Auswirkung auf den CO2 Fußabdruck eine Rolle spielen. 

Im Betrieb können Heimspeicher sehr unterschiedlich agieren - je nach Steuerungsvorgabe durch den Systemhersteller bzw. teilweise auch durch nutzer-spezifische Vorgabe. Die Betreibsstrategie hat einen wesentlichen Einfluss auf z.B. die Speicherperformance, die Systemalterung und  auch auf das ölkonomische Potential.

Im Notebook soll folgendes untersucht werden:
- Durchführung einer Zeitreihensimulation für einen stationären Batteriespeicher
- Techno-ökonomische Analyse des Anwendungsfalls "Heimspeicher"
- Auswirkungen unterschiedlicher Betreibsstrategien auf die Systemperformance

In [ ]:
# Import wichtiger Packages - nicht ändern, Codeblock unbedingt ausführen!
import pandas as pd      # Das pandas Paket dient der Datenanalyse - siehe: https://pandas.pydata.org/

In [ ]:
# Einstellungen zu Darstellungen "Plotting", optional aber hilfreich
pd.options.plotting.backend = "plotly"
template = "plotly_white"
# template = "plotly_dark"

## Datensatz für Heimspeicheranwendung (und weitere)
Für unsere Analyse werfen wir einen Blick in einen *repräsentativen* Haushalt mit einer Solar-PV-Anlage. Die Daten stammen aus der Publikation [*Standard Battery Application Profiles (SBAP) paper*](https://doi.org/10.1016/j.est.2019.101077), welches Anwendungs- und Speicher-Betreibsprofile mehrerer Beispielanwendungen bereitstellt - alle dort genutzen Profildaten können als "Open Data" verwendet werden! Bei Verswendung von Excel / Matlab: sie finden Diese Daten im CSV Format unter folgender Adresse:  https://gitlab.com/HesseHSKempten/23s_statbat/-/blob/main/data/household_profile.csv


In [ ]:
profile = pd.read_csv("../data/household_profile.csv", index_col=0, parse_dates=True) # Einlesen des Profils aus der CSV Datei
profile.plot(template=template, labels={"value": "Power [kW]"}) # Darstellung der Profildaten mit der "Plotly" Bibilothek

Werfen wir zunächst einen Blick auf unser Basisszenario - ein System ohne Batterie-Energiespeicher:

<div class="alert alert-block alert-info">
<b>Task I</b> Analysieren Sie die Verbrauchs- und Erzeugungsprofile:
<ol>
    <li> Berechnen Sie die Residuallast, speichern Sie diese als neue Spalte im Dataframe "profile" und visualisieren Sie diese als Grafik </li>
    <li> Ermitteln Sie den gesamten Energieverbrauch und die Erzeugung in einem Jahr und berechnen Sie die daraus resultierenden Stromkosten. Betrachten Sie 0,35 €/kWh für den Strompreis und 0,07 €/kWh für die Einspeisevergütung </li>
    <li> Implementieren Sie die Fun ktion "self_cponsumption_rate" , welche den Eigenverbrauch berechnet </li>
    <li> Implementieren Sie die Fun ktion "self_sufficiency_rate" , welche den Autarkiegrad berechnet </li>
    <li> Verwenden Sie diese beiden Funktionen, um Werte für oben gennate Stromtarife zu berechnen.
</ol>
</div>

<div class="alert alert-block alert-warning">
<b>Achtung!</b> Achten Sie auf passende Unterscheidung zwischen "Leistung" und "Enerige" - berücksichtigen Sie die Zeitauflösung der Daten von 15 Minuten (!)
</div>

In [ ]:
# Task I.1: Berechnen Sie die Residuallast, speichern Sie diese als neue Spalte im Dataframe "profile" und visualisieren Sie diese als Grafik
profile["residual"] = profile['load'] - profile['pv']
profile.plot(template=template, labels={"value": "Power [kW]"}) # Darstellung der Profildaten mit der "Plotly" Bibilothek

In [ ]:
# Task I.2: Ermitteln Sie den gesamten Energieverbrauch und die Erzeugung in einem Jahr und berechnen Sie die daraus resultierenden Stromkosten. Betrachten Sie 0,35 €/kWh für den Strompreis und 0,07 €/kWh für die Einspeisevergütung

In [ ]:
# Task I.3: Implementieren Sie eine Funktion "self_consumption_rate" , welche Eigenverbrauchsanteil berechnet
def self_consumption_rate(...)



In [ ]:
# Task I.4: Implementieren Sie die Fun ktion "self_sufficiency_rate" , welche den Autarkiegrad berechnet

In [ ]:
# Task I.5: Verwenden Sie diese beiden Funktionen, um Werte für oben gennate Stromtarife zu berechnen.

## Greedy strategy

Um die Sonnenenergie besser zu nutzen, entscheidet sich der Haushalt für die Installation eines Speichersystems mit  5 kWh Nennleistung und 5 kW max. Leistung. Allerdings wird die Batterie "die Arbeit nicht für sich selbst erledigen ..."  - Wir müssen vorgeben, was sie tun soll!

Für unseren ersten Versuch nutzen wir die einfachste Strategie zur **Verbesserung des Eigenverbrauchs** : Der Speicher prüft die Residuallast  - sobald die PV-Erzeugung die aktuelle Last übersteigt, lädt er die Batterie. Sobald die PV-Erzeugung hingegen unter den Bedarf fällt, entlädt sich die Batterie wieder, um die Differenz auszugleichen. Diese *simple* Strategie versucht, die gesamte überschüssige lokal erzeugte Energie zwischenzuspeichern, sie wird als *greedy* Strategie bezeichnet.

<div class="alert alert-block alert-info">
<b>Task II</b> Implementieren und Analysieren Sie eine Greedy Strategie für den Heimspeicher
<ol>
    <li> Implementieren Sie die Greedy Strategie zur Steigerung des Eigenverbrauchs. Definieren Sie eine Funktion, welche das "Dataframe" "profile" sowie maximale Kapazität & maximale Leistung des Speichers als Eingangsgrößen übernimmt und ein neues Dataframe (Kopie!) mit  Batterie-SOC, der Leistung des Speichers und der resultierenden Netzleistung für jeden Zeitschritt zurückgibt. Prüfen Sie während des gesamten Betriebs, dass die Leistungs- und SOC-Grenzwerte nicht verletzt werden. Nehemn Sie Systemverluste mit Lade-/Entladeeffizienzen von 95 % an.</li>
    <li> Nehmen Sie Beispielwerte für Kapazität und Leistung an und berechnen Sie die sich nun ergebende Eigenverbrauchsquote und den Autraktiegrad des Haushalts mit Speicher. Vergelichen Sie die Ergebnisse mirt der zuvor analysierten Situation ohne Speicher. </li>
    <li> Berechnen Sie die nun auftretenden Stromkosten und ihre <i>Ersparnisse</i> gegenüber dem Referenzszenario. </li>
</ol>
</div>

<div class="alert alert-block alert-warning">
<b>Hint!</b> Durchlaufen Sie das Dataframe, um die Leistung und den SOC in jedem Zeitschritt zu berechnen. Befolgen Sie die Konvention von positiver Leistung zum Laden und negativer Leistung zum Entladen. Beachten Sie die richtige Umrechnung von Leistung zu Energie

</div>

In [ ]:
# Task II.1: Implementieren Sie die Greedy Strategie zur Steigerung des Eigenverbrauchs. 
    # Definieren Sie eine Funktion, welche das "Dataframe" "profile" sowie maximale Kapazität & maximale Leistung des Speichers als Eingangsgrößen übernimmt 
    # und ein neues Dataframe (Kopie!) mit  Batterie-SOC, der Leistung des Speichers und der resultierenden Netzleistung für jeden Zeitschritt zurückgibt. 
    # Prüfen Sie während des gesamten Betriebs, dass die Leistungs- und SOC-Grenzwerte nicht verletzt werden. Nehemn Sie Systemverluste mit Lade-/Entladeeffizienzen von 95 % an.

def greedy_strategy(profile, capacity, max_power, eff=0.95, initial_soc=0.0):
    df = profile.copy() # make a new copy of the dataframe
    # add new empty columns to the dataframe
    df["grid"] = 0.0   # grid power in kW
    df["power"] = 0.0  # battery power in kW
    df["soc"]  = 0.0   # battery SOC in p.u.

        ...

Let's simulate the operation and visualize the results.

In [ ]:
# Nehmen Sie Beispielwerte für Kapazität (Wert zwischen 1 und 20 kWh) und Leistung an:
capacity = 5.0 # kWh
max_power = 5.0 # kW

In [ ]:
df_greedy = greedy_strategy(profile=profile, capacity=capacity, max_power=max_power) # this can take some time to compute

In [ ]:
df_greedy[["residual", "power"]].plot(template=template, labels={"value": "Power [kW]"})# .update_yaxes(autorange="reversed")

In [ ]:
df_greedy["soc"].plot(template=template, labels={"value": "SOC"})

In [ ]:
# TASK II.2: Berechnen Sie die sich nun ergebende Eigenverbrauchsquote und den Autraktiegrad des Haushalts mit Speicher. 
# Vergelichen Sie die Ergebnisse mirt der zuvor analysierten Situation ohne Speicher.


In [ ]:
# TASK II.3: Berechnen Sie die nun auftretenden Stromkosten und ihre Ersparnisse gegenüber dem Referenzszenario.


## Feed-in damp Strategie

Lassen Sie uns eine aus Stomnetz und Batteriealterungs-Sicht "verbesserte" Strategie entwerfen: die "Feed-in Damp" Strategie

Im Falle der Greedy strategie füllt sich die Batterie insbesondere im Sommer bereits früh am Tage. So kann die Batterie Einspeisespitzen der PV-Analge (Rückspeisung ins Stromnetz) nicht effektiv verhindern - ein Netzausbau wird so schneller benötigt. Wir wollen daher die im Paper [Zeh & Witzman](https://doi.org/10.1016/j.egypro.2014.01.164) vorgestellte *feed-in damp* Strategie umsetzen. Nutzen sie (wie im Greedy Beispeil) eine Überprüfung der Residuallast, und laden bzw. entladen Sie den Speicher exakt wie zuvor diskutiert (Greedy Strategie). Nun soll jedoch der Speicher erst genau eine Stunde vor Sonnenuntergang zum Limit gefüllt werden  - die entsprechende Ladeleistung ergibt sich wie folgt:

$$ P_{max, damp} = \frac{E_{remaining}}{t_{sunset} - t_{now} - 1 \text{ h}} $$

Lassen Sie uns prüfen, ob Netzentlasung und Eigenverbrauchs-Erhöhung parallel gut möglich sind, soweit die Prognosedaten passend vorliegen.

Die Schwierigkeit dieser Aufgabe liegt darin die Sunnenuntergangszeit für den jew. Tag und unseren Anlagenstandort passend zu berechnen. Zum Glück können wir uns an der enorm großen Anzahl an freien Programmbibliotheken ("Package") für Python bedienen.  Wir wählen hier das Package [*suntime*](https://github.com/SatAgro/suntime)

In [ ]:
# documentation: https://github.com/SatAgro/suntime
from suntime import Sun
from datetime import timedelta, timezone

Im folgenden ein Beispiel zur Nutzung von suntime

In [ ]:
# calculate sunset time for a particular location and date
latitude = 47.7 # Daten für Kempten
longitude = 10.31 # Daten für Kempten
sun = Sun(latitude, longitude) 

# suntime returns time in UTC, so it has to be first translated to the local timezone
sunset_time_utc   = sun.get_sunset_time()
sunset_time_local = sunset_time_utc.astimezone(timezone(timedelta(hours=+1), name="Europe/Berlin")) # "Europe/Berlin" timezone
print(f"UTC time:   ", sunset_time_utc)
print(f"Local time: ", sunset_time_local)

In [ ]:
# calculate remainig time for the sunset
day = profile.index[1]
sunset_time = sun.get_sunset_time(day).replace(tzinfo=None) # delete timezone metadata to allow datetime-arithmetic
offset = sunset_time - day
print(offset)
print(offset.seconds, "s") # offset in seconds

<div class="alert alert-block alert-info">
<b>Task III</b> Implementieren und analysieren Sie die feed-in damp Strategie:
<ol>
    <li> Immplementieren Sie die "Feed-In Damping Strategie (FID)"  für die Eigenverbrauchserhöhung. Erstellen Sie eine Funktion, die das "profile" dataframe sowie Speicherleistung und -Kapazität einliest und ein neues Dataframe mit den Zeitreihen für SOC und Batterieleistung ausgibt. Überprüfen Sie im gesamten Betreib, dass Leistung und SOC Limits nicht verletzt werden. Die Lade-/Entladeverluste sind abermals auf  95% zu setzen. </li>
    <li> Berechnen Sie Eigenverbrauch und Autarkiegrad der FID Strategie. Vergleichen Sie zum Basisszenario. </li>
    <li> Berechnen Sie die Elektrizitätskosten und die <i>Ersparnisse</i> ggü. dem Referenzfall. </li>
</ol>
</div>

<div class="alert alert-block alert-warning">
<b>Hinweis!</b> Nutzen Sie <i> suntime </i> zur Prognose des Sonnenuntergangs an einem jeweiligen Tag
</div>

In [ ]:
# Task III.1 Immplementieren Sie die "Feed-In Damping Strategie (FID)"  für die Eigenverbrauchserhöhung. 
# Erstellen Sie eine Funktion, die das "profile" dataframe sowie Speicherleistung und -Kapazität einliest und 
# ein neues Dataframe mit den Zeitreihen für SOC und Batterieleistung ausgibt.
# Überprüfen Sie im gesamten Betrieb, dass Leistung und SOC Limits nicht verletzt werden. 
# Die Lade-/Entladeverluste sind abermals auf  95% zu setzen.
def feedin_damp(profile, capacity, max_power, eff=0.95, initial_soc=0.0):
    df = profile.copy() # make a new copy of the dataframe
    # add new empty columns to the dataframe
    df["grid"] = 0.0   # grid power in kW
    df["power"] = 0.0  # battery power in kW
    df["soc"]  = 0.0   # battery SOC in p.u.
    
    # task
    ...

In [ ]:
df_damp = feedin_damp(profile, capacity=capacity, max_power=max_power) # this will also take some time to compute

In [ ]:
df_damp[["residual", "grid", "power"]].plot(template=template)

In [ ]:
# task III.2: Berechnen Sie Eigenverbrauch und Autarkiegrad der FID Strategie. Vergleichen Sie zum Basisszenario.

In [ ]:
# task III.3: BBerechnen Sie die Elektrizitätskosten und die Ersparnisse ggü. dem Referenzfall.

### Vergelich *Greedy* vs. *Feed-in Damp*

Nachdem beide Strategien impementiert wurden, lassen sich noch zahlreiche Analysen und Darstellungen durchführen - wir werden diese in der Vorlesung aufgreifen...

In [ ]:
dfbat = pd.concat([df_greedy["power"].rename("power - greedy"), df_damp["power"].rename("power - feed-in damp")], axis=1)
dfbat.plot(template=template, labels={"value": "Power [kW]"})

In [ ]:
dfbat.plot.hist(template=template, log_y=True, labels={"value": "Power [kW]"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

In [ ]:
dfsoc = pd.concat([df_greedy["soc"].rename("soc - greedy"), df_damp["soc"].rename("soc - feed-in damp")], axis=1)
dfsoc.plot(template=template, labels={"value": "SOC"})

In [ ]:
dfsoc.plot.hist(template=template, log_y=True, labels={"value": "SOC"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

In [ ]:
dfgrid = pd.concat([df_greedy["grid"].rename("soc - greedy"), df_damp["grid"].rename("soc - feed-in damp")], axis=1)
dfgrid.plot(template=template, labels={"value": "Power [kW]"})

In [ ]:
dfgrid.plot.hist(template=template, log_y=True, labels={"value": "Grid power [kW]"}).update_layout(barmode='overlay').update_traces(opacity=0.75)

<div class="alert alert-block alert-info">
<b> Task IV </b> Vergelichen Sie die Strategien - Listen Sie vor und Nachteile der jew. Strategien auf, aus Perspektive des Netzbetreibers und auch aus Sicht des Eigentümers.
</div>

1. Vorteile von Greedy ggü. FID aus Sicht des Eigentümers:
- 
- 
- 
2. Nachteile von Greedy ggü. FID aus Sicht des Eigentümers:
- 
- 
- 
3. Vorteile von Greedy ggü. FID aus Sicht des Netzbetreibers:
- 
- 
- 
4. Nachteile von Greedy ggü. FID aus Sicht des Netzbetreibers:
- 
- 
- 
